<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AutoGen Agents with Retrieval Augmented Generation


## Use Case

In this tutorial, I will create the retrieval augmented agents with the following document:

[RETRIEVAL AUGMENTED GENERATION AND REPRESENTATIVE
VECTOR SUMMARIZATION FOR LARGE UNSTRUCTURED
TEXTUAL DATA IN MEDICAL EDUCATION](https://arxiv.org/pdf/2308.00479.pdf)

You should be able to see the agents are able to perform retrieval augmented generation based on the document above and answer question.


### Environment Preparation


In [ ]:
%pip install pyautogen[retrievechat] langchain "chromadb<0.4.15" -q

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

### Steps


#### 1. Configure Embedding Function

We will use OpenAI embedding function.


In [ ]:
from chromadb.utils import embedding_functions

openai_embedding_function = embedding_functions.OpenAIEmbeddingFunction(
    api_key=config_list[0]["api_key"]
)

#### 2. Configure Text Splitter

LangChain has done a great job in text splitting, so we will use its components.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\r", "\t"])

#### 3. Configure Vector Store

By default, the AutoGen retrieval augmented agents use `chromadb` as vector store.

Developers can configure preferred vector store by extending the class `RetrieveUserProxyAgent` and overriding function `retrieve_docs`.

AutoGen also supports simple configuration items to customize Chromadb storage.

In this demo, we will specify the collection name by `retreive_config` item `collection_name`. You should be able to see it in step 4.


#### 4. Create Retrieval Augmented Agents


In [ ]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

llm_config = {"request_timeout": 600, "config_list": config_list, "temperature": 0}

assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)

rag_agent = RetrieveUserProxyAgent(
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": "./rag.pdf",
        "collection_name": "rag_collection",
        "embedding_function": openai_embedding_function,
        "custom_text_split_function": text_splitter.split_text,
    },
)

### It's time to start a chat with the RAG agent.


In [ ]:
assistant.reset()
rag_agent.initiate_chat(
    assistant, problem="What is the workflow in docGPT?", n_results=2
)